In [1]:
import sys
sys.path.append('/Users/nmadd/Dropbox (Personal)/code/coin-metrics/research-infrastructure/')
%matplotlib inline
import pandas as pd
import math


In [2]:
from modules.Reports.CoinbaseReport import CoinbaseReport
from modules.Reports.ReportGenerator import ReportGenerator
from modules.Pipelines.NetworkDataPipe import NetworkDataPipe
from modules.Visualizers.LineChart import LineChart
from modules.Visualizers.SubPlot import SubPlot
from modules.Visualizers.Table import Table
from modules.Transformers.Divider import Divider
from modules.Transformers.Multiplier import Multiplier
from modules.DataLoaders.CM_API import CM_API

ModuleNotFoundError: No module named 'modules.Visualizers.LineChartMetricByAssets'

In [ ]:
api_key = 'sIxXFWu6DsRlghV3XKL5'
start = '2020-01-20'
end = None

In [ ]:
_assets = [
    'btc',
    'xrp',
    'eth',
    'bch',
    'ltc',
    'xlm',
    'xtz',
    'bsv',
    'ada',
    'zec',
    'link',
    'dcr',
    'mkr',
    'bat',
    'usdt',
    'usdt_eth',
    'pax',
    'usdc'
]

assets = _assets

In [ ]:
### Metrics ###

valuation = [
    'CapRealUSD',
    'CapMrktCurUSD',
    'CapMVRVCur',
#     'NVTAdj',
    'VtyDayRet30d',
    'PriceUSD'  
]

valuation_table = [
    'CapRealUSD',
#     'CapMrktCurUSD',
    'CapMVRVCur',
#     'NVTAdj',
    'VtyDayRet30d',
    'PriceUSD'  
]

economics = [
    'TxCnt',
    'TxTfrCnt',
    'TxTfrValAdjUSD',
    'IssContPctAnn',
    'FeeMedUSD',
#     'VelActAdj1yr',
#     'VelCurAdj1yr',
#     'TxTfrValAdjByte'
]

usage = [
    'AdrActCnt',
    'AdrBalUSD10Cnt',
    'AdrBalUSD1MCnt',
    'BlkSizeByte'
]

# 'FeesToBlockReward', 'DeltaCap'
security_and_health = [
        'HashRate',
#         'DiffMean',
        'FeeTotUSD',
        'FeeRevPct',
        'RevUSD'
#         'RevAllTimeUSD'
]

extra= [
    'SplyCur',
    'SplyAct30d',
]


metrics = valuation + economics + usage + security_and_health + extra

In [ ]:
_api = CM_API()
asset_info = _api.get_coinmetrics_discovery_asset_info(api_key=api_key)
asset_info = asset_info['assetsInfo']
asset_info_dict = {}

df = pd.DataFrame()

for asset in asset_info:
    asset_id = asset['id']
    asset_metrics = asset['metrics']
    if 'SplyAct30d' in asset_metrics:
        asset_metrics.append('SplyAct30dPercent')
    asset_info_dict[asset_id] = asset_metrics


# create master df by checking one by one if assets avaialble 
for asset in assets:
    for metric in metrics:
        if metric in asset_info_dict[asset]:
            metric_df = NetworkDataPipe(api_key=api_key, metrics=[metric], assets=[asset], start=start, end=end).run()
            df = _api.concat_dataframes([df, metric_df])

df.head()

In [ ]:
# asset lists
valuation_assets = [
    'btc',
    'xrp',
    'eth',
    'bch',
    'ltc',
    'xlm',
    'xtz',
    'bsv',
    'ada',
    'zec',
    'link',
    'dcr',
    'mkr',
    'bat',
    'usdt',
    'usdt_eth',
    'pax',
    'usdc'
]

economic_assets = [
    'btc',
    'xrp',
    'eth',
    'bch',
    'ltc',
    'xlm',
    'xtz',
    'bsv',
    'ada',
    'zec',
#     'link',
#     'dcr',
#     'mkr',
#     'bat',
#     'usdt',
#     'usdt_eth',
#     'pax',
#     'usdc'
]

usage_assets = [
    'btc',
    'xrp',
    'eth',
    'bch',
    'ltc',
    'xlm',
    'xtz',
    'bsv',
    'ada',
    'zec',
    'link',
#     'dcr',
#     'mkr',
#     'bat',
#     'usdt',
#     'usdt_eth',
#     'pax',
#     'usdc'
]

security_assets = [
    'btc',
    'eth',
    'bch',
    'ltc',
    'bsv',
    'zec',
#     'xrp',
#     'xlm',
#     'xtz',
#     'ada',
#     'link',
#     'dcr',
#     'mkr',
#     'bat',
#     'usdt',
#     'usdt_eth',
#     'pax',
#     'usdc'
]


In [ ]:
for asset in assets:
    if 'SplyAct30d' in asset_info_dict[asset]:
        df = Divider(column_a=f'{asset}.SplyAct30d', column_b=f'{asset}.SplyCur', new_column_name=f'{asset}.SplyAct30dPercent').transform(df)
        df = Multiplier(column=f'{asset}.SplyAct30dPercent', multiplier=100).transform(df)

In [ ]:
def map_metric_name(metric):
    metric_mappings = {
        'Asset': 'Asset',
        'CapRealUSD' : 'Realized Cap',
        'CapMrktCurUSD': 'Market Cap',
        'CapMVRVCur': 'Market Value to Realized Value (MVRV) ',
        'VtyDayRet30d': 'Volatility, 30d',
        'PriceUSD': 'Price, USD',
        'AdrActCnt': 'Active Addresses',
        'AdrBalUSD10Cnt': 'Addresses with Balance >= $10',
        'AdrBalUSD1MCnt': 'Addresses with Balance >= $1M',
        'SplyAct30d': 'Active Supply, 30d',
        'SplyAct30dPercent' : 'Active Supply Percentage, 30d',
        'BlkSizeByte': 'Block Size, Bytes',
        'TxCnt': 'Transaction Count',
        'TxTfrCnt': 'Transfer Count',
        'TxTfrValAdjUSD': 'Adjusted Transfer Value',
        'IssContPctAnn': 'Issuance, Annualized',
        'FeeMedUSD': 'Median Transaction Fee',
        'HashRate': 'Hash Rate',
        'DiffMean': 'Difficulty, Mean',
        'FeeTotUSD': 'Fees, USD',
        'FeeRevPct': 'Fee to Revenue Percentage',
        'RevUSD': 'Miner Revenue, USD'
        
    }

    return metric_mappings[metric] if metric in list(metric_mappings.keys()) else metric

In [ ]:
growth = True
seven_day_rolling_average = False
layout_type = 'growth'
y2_price_trace = False

subplot_visuals = []
number_of_assets = 0
for metric in metrics + ['SplyAct30dPercent']:
    metric_title = map_metric_name(metric)
    for value in asset_info_dict.items():
        if value[0] in assets and metric in value[1]:
            number_of_assets += 1
    _number_of_rows = math.ceil(number_of_assets / 3) 
    _number_of_rows = _number_of_rows if _number_of_rows > 0 else 1
    #### filter out xlm and xrp ###
#     assets = filter(lambda x: x not in ['xrp', 'xlm'], assets)
    _subplot = SubPlot(df=df, title=f'{metric_title}', layout_type= layout_type, number_of_rows=_number_of_rows, number_of_columns=3, columns_to_plot=[f'{asset}.{metric}' for asset in assets if metric in asset_info_dict[asset]], growth=growth, seven_day_rolling_average=seven_day_rolling_average, y2_price_trace=y2_price_trace)
    _visual = _subplot.run()
    _height = math.ceil(number_of_assets / 3) * 150
    _height = _height if _height > 0 else 150
    _height = _height if _height > 300 else 500
    _visual.fig.update_layout(
        showlegend=False,
        height= _height,
        width = 800,
        legend = {
#         'orientation': 'v',
#         'xanchor': 'left',
        'xanchor': 'center',
        'yanchor': 'bottom',
        'x': .5,
        'y': -.25,
        'orientation': 'h'
        })
#     _visual.export(title=visual.title, repo='2019_Year_In_Review')

    if layout_type == 'growth' or layout_type == 'volatility':
        _visual.fig.update_layout(
            yaxis = {'tickformat': ',.0%', 'side': 'left'},
            xaxis = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis2 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis2 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis3 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis3 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis4 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis4 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis5 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis5 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis6 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis6 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis7 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis7 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis8 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis8 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis9 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis9 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis10 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis10 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis11 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis11 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis12 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis12 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis13 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis13 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis14 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis14 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis15 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis15 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis16 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis16 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis17 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis17 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis18 = {'tickformat': ',.0%', 'side': 'left'},
            xaxis18 = {
                'showgrid' : False
            }
         )
        
    if layout_type == 'raw_percent':
        _visual.fig.update_layout(
            yaxis1 = {'ticksuffix': '%', 'side': 'left'},
            xaxis1 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis2 = {'ticksuffix': '%', 'side': 'left'},
            xaxis2 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis3 = {'ticksuffix': '%', 'side': 'left'},
            xaxis3 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis4 = {'ticksuffix': '%', 'side': 'left'},
            xaxis4 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis5 = {'ticksuffix': '%', 'side': 'left'},
            xaxis5 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis6 = {'ticksuffix': '%', 'side': 'left'},
            xaxis6 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis7 = {'ticksuffix': '%', 'side': 'left'},
            xaxis7 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis8 = {'ticksuffix': '%', 'side': 'left'},
            xaxis8 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis9 = {'ticksuffix': '%', 'side': 'left'},
            xaxis9 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis10 = {'ticksuffix': '%', 'side': 'left'},
            xaxis10 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis11 = {'ticksuffix': '%', 'side': 'left'},
            xaxis11 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis12 = {'ticksuffix': '%', 'side': 'left'},
            xaxis12 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis13 = {'ticksuffix': '%', 'side': 'left'},
            xaxis13 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis14 = {'ticksuffix': '%', 'side': 'left'},
            xaxis14 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis15 = {'ticksuffix': '%', 'side': 'left'},
            xaxis15 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis16 = {'ticksuffix': '%', 'side': 'left'},
            xaxis16 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis17 = {'ticksuffix': '%', 'side': 'left'},
            xaxis17 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis18 = {'ticksuffix': '%', 'side': 'left'},
            xaxis18 = {
                'showgrid' : False
            }
         )
    if layout_type == 'raw_value':
        _visual.fig.update_layout(
            yaxis2 = {'side': 'left'},
            xaxis2 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis3 = {'side': 'left'},
            xaxis3 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis4 = {'side': 'left'},
            xaxis4 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis5 = {'side': 'left'},
            xaxis5 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis6 = {'side': 'left'},
            xaxis6 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis7 = {'side': 'left'},
            xaxis7 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis8 = {'side': 'left'},
            xaxis8 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis9 = {'side': 'left'},
            xaxis9 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis10 = {'side': 'left'},
            xaxis10 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis11 = {'side': 'left'},
            xaxis11 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis12 = {'side': 'left'},
            xaxis12 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis13 = {'side': 'left'},
            xaxis13 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis14 = {'side': 'left'},
            xaxis14 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis15 = {'side': 'left'},
            xaxis15 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis16 = {'side': 'left'},
            xaxis16 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis17 = {'side': 'left'},
            xaxis17 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis18 = {'side': 'left'},
            xaxis18 = {
                'showgrid' : False
            }
         )
        
    if layout_type == 'fees':
        _visual.fig.update_layout(
            yaxis = {'tickprefix': '$', 'side': 'left'},
            xaxis = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis2 = {'tickprefix': '$', 'side': 'left'},
            xaxis2 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis3 = {'tickprefix': '$', 'side': 'left'},
            xaxis3 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis4 = {'tickprefix': '$', 'side': 'left'},
            xaxis4 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis5 = {'tickprefix': '$', 'side': 'left'},
            xaxis5 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis6 = {'tickprefix': '$', 'side': 'left'},
            xaxis6 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis7 = {'tickprefix': '$', 'side': 'left'},
            xaxis7 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis8 = {'tickprefix': '$', 'side': 'left'},
            xaxis8 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis9 = {'tickprefix': '$', 'side': 'left'},
            xaxis9 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis10 = {'tickprefix': '$', 'side': 'left'},
            xaxis10 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis11 = {'tickprefix': '$', 'side': 'left'},
            xaxis11 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis12 = {'tickprefix': '$', 'side': 'left'},
            xaxis12 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis13 = {'tickprefix': '$', 'side': 'left'},
            xaxis13 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis14 = {'tickprefix': '$', 'side': 'left'},
            xaxis14 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis15 = {'tickprefix': '$', 'side': 'left'},
            xaxis15 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis16 = {'tickprefix': '$', 'side': 'left'},
            xaxis16 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis17 = {'tickprefix': '$', 'side': 'left'},
            xaxis17 = {
                'showgrid' : False
            }
         )
        _visual.fig.update_layout(
            yaxis18 = {'tickprefix': '$', 'side': 'left'},
            xaxis18 = {
                'showgrid' : False
            }
         )
        
        

#     _visual.export(title=metric_title, repo='2019_Year_In_Review')
    _visual.show()
       
    number_of_assets = 0

In [ ]:
table_data= df.loc['2019-12-30':'2019-12-30']
test_data = df.loc['2019-01-01':'2019-12-30']
test_avg = test_data.mean(axis=0)
test_avg = test_avg.to_frame().T
test_avg


In [ ]:
def create_year_end_table(df, title, assets, metrics, column_order):
    table_df = pd.DataFrame(columns=['Asset'])

    for asset in assets:
        _new_row = {}
        _new_row['Asset'] = asset
        for metric in metrics:
            _new_row[metric] = df.iloc[0][f'{asset}.{metric}'] if metric in asset_info_dict[asset] else '-'
            _new_row['CapMrktCurUSD'] = df.iloc[0][f'{asset}.CapMrktCurUSD']
        table_df = table_df.append(_new_row, ignore_index=True)
    table_df.sort_values(by='CapMrktCurUSD', ascending=False, inplace=True)
    if len(list(table_df.columns)) > 5:
        table_df.drop('CapMrktCurUSD', axis=1, inplace=True)
    if 'VtyDayRet30d' in list(table_df.columns):
        table_df['VtyDayRet30d'] = table_df['VtyDayRet30d'] * 100
    table_df.fillna('-', inplace=True)
    
        
    mapper =  {
       'AdrActCnt': '{0:,.0f}',
       'AdrBalUSD10Cnt': '{0:,.0f}',
       'AdrBalUSD1MCnt': '{0:,.0f}',
#        'SplyAct30d': '{0:,.2f}',
       'BlkSizeByte': '{0:,.0f}',
       'SplyAct30dPercent': '{0:.2f}%',
       'CapMVRVCur': '{0:,.2f}',
       'CapMrktCurUSD': '${:,.2f}B',
       'CapRealUSD': '${:,.2f}B',
       'PriceUSD': '${:,.2f}',
       'VtyDayRet30d': '{:,.2f}%',
       'TxCnt': '{:,.0f}',
       'TxTfrCnt': '{:,.0f}',
       'TxTfrValAdjUSD': '${:,.2f}M',
       'FeeMedUSD': '${:,.4f}',
       'IssContPctAnn': '{0:.2f}%',
       'HashRate': '{0:,.0f}',
       'DiffMean': '{0:,.0f}',
       'FeeTotUSD': '${:,.0f}',
       'FeeRevPct': '{0:.2f}%',
       'RevUSD': '${:,.0f}'
        
    }
    
    if 'CapMrktCurUSD'in list(table_df.columns):
        table_df['CapMrktCurUSD'] = table_df['CapMrktCurUSD'].apply(lambda x: x / 1000000000 if x != '-' else x)
    if 'CapRealUSD'in list(table_df.columns):
        table_df['CapRealUSD'] = table_df['CapRealUSD'].apply(lambda x: x / 1000000000 if x != '-' else x)
    if 'TxTfrValAdjUSD'in list(table_df.columns):
        table_df['TxTfrValAdjUSD'] = table_df['TxTfrValAdjUSD'].apply(lambda x: x / 1000000 if x != '-' else x)

    for key, value in mapper.items():
        if key in list(table_df.columns):
            table_df[key] = table_df[key].apply(lambda x: value.format(x) if x != '-' else x)
            

    
    table_df.columns = map(map_metric_name, list(table_df.columns))
    
    
    
    _table = Table(df=table_df, title=None, table_headers=list(table_df.columns), table_columns=list(table_df.columns), column_order=column_order).run()
    _table.fig.update_layout(height=475, margin={'l':50, 'r':50, 'b': 25, 't': 25,})
#     _table.export(title=title, repo='2019_Year_In_Review')
    return _table.show()

create_year_end_table(df=test_avg, title='Usage and Adoption Summary', assets=assets, metrics=usage + ['SplyAct30dPercent'], column_order=[0,1,2,3,5,4])
create_year_end_table(df=table_data, title='Valuation Summary', assets=assets, metrics=valuation_table, column_order=[0,4,2,3,1,5])
create_year_end_table(df=test_avg, title='Economics Summary', assets=assets, metrics=economics, column_order=[0,5,4,1,2,3])
create_year_end_table(df=test_avg, title='Security and Health Summary', assets=assets, metrics=list(filter(lambda x: x != 'Market Cap', security_and_health)), column_order=[0, 3, 4, 1, 2])

In [ ]:
median_tx_subplot = SubPlot(df=df, title=f'{metric_title}', layout_type= layout_type, number_of_rows=_number_of_rows, number_of_columns=3, columns_to_plot=[f'{asset}.{metric}' for asset in assets if metric in asset_info_dict[asset]], growth=growth, seven_day_rolling_average=seven_day_rolling_average, y2_price_trace=y2_price_trace)

In [ ]:
median_tx_subplot = SubPlot(df=df, title='Median Transaction Fee', layout_type= 'fees', number_of_rows=3, number_of_columns=3, columns_to_plot=[f'{asset}.FeeMedUSD' for asset in ['btc', 'eth', 'bch', 'ltc', 'bsv', 'ada', 'zec', 'dcr'] if 'FeeMedUSD' in asset_info_dict[asset]], growth=False, seven_day_rolling_average=True)
median_visual = median_tx_subplot.run()

median_visual.fig.update_layout(
    yaxis = {'tickprefix': '$', 'side': 'left'},
    xaxis = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis2 = {'tickprefix': '$', 'side': 'left'},
    xaxis2 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis3 = {'tickprefix': '$', 'side': 'left'},
    xaxis3 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis4 = {'tickprefix': '$', 'side': 'left'},
    xaxis4 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis5 = {'tickprefix': '$', 'side': 'left'},
    xaxis5 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis6 = {'tickprefix': '$', 'side': 'left'},
    xaxis6 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis7 = {'tickprefix': '$', 'side': 'left'},
    xaxis7 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis8 = {'tickprefix': '$', 'side': 'left'},
    xaxis8 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis9 = {'tickprefix': '$', 'side': 'left'},
    xaxis9 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis10 = {'tickprefix': '$', 'side': 'left'},
    xaxis10 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis11 = {'tickprefix': '$', 'side': 'left'},
    xaxis11 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis12 = {'tickprefix': '$', 'side': 'left'},
    xaxis12 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis13 = {'tickprefix': '$', 'side': 'left'},
    xaxis13 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis14 = {'tickprefix': '$', 'side': 'left'},
    xaxis14 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis15 = {'tickprefix': '$', 'side': 'left'},
    xaxis15 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis16 = {'tickprefix': '$', 'side': 'left'},
    xaxis16 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis17 = {'tickprefix': '$', 'side': 'left'},
    xaxis17 = {
        'showgrid' : False
    }
 )
median_visual.fig.update_layout(
    yaxis18 = {'tickprefix': '$', 'side': 'left'},
    xaxis18 = {
        'showgrid' : False
    }
 )

median_visual.fig.update_layout(
    showlegend=False,
    height= 500,
    width = 800,
    legend = {
#         'orientation': 'v',
#         'xanchor': 'left',
    'xanchor': 'center',
    'yanchor': 'bottom',
    'x': .5,
    'y': -.25,
    'orientation': 'h'
    })


In [ ]:
df[['ada.IssContPctAnn', 'xlm.IssContPctAnn']]
issuance_subplot = SubPlot(df=df, title='Issuance, Annualized', layout_type= 'raw_percent', number_of_rows=3, number_of_columns=3, columns_to_plot=[f'{asset}.IssContPctAnn' for asset in ['btc', 'eth', 'bch', 'ltc', 'xtz', 'bsv', 'zec'] if 'IssContPctAnn' in asset_info_dict[asset]], growth=False, seven_day_rolling_average=True)
issuance_visual = issuance_subplot.run()

issuance_visual.fig.update_layout(
    yaxis = {'ticksuffix': '%', 'side': 'left'},
    xaxis = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis2 = {'ticksuffix': '%', 'side': 'left'},
    xaxis2 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis3 = {'ticksuffix': '%', 'side': 'left'},
    xaxis3 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis4 = {'ticksuffix': '%', 'side': 'left'},
    xaxis4 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis5 = {'ticksuffix': '%', 'side': 'left'},
    xaxis5 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis6 = {'ticksuffix': '%', 'side': 'left'},
    xaxis6 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis7 = {'ticksuffix': '%', 'side': 'left'},
    xaxis7 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis8 = {'ticksuffix': '%', 'side': 'left'},
    xaxis8 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis9 = {'ticksuffix': '%', 'side': 'left'},
    xaxis9 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis10 = {'ticksuffix': '%', 'side': 'left'},
    xaxis10 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis11 = {'ticksuffix': '%', 'side': 'left'},
    xaxis11 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis12 = {'ticksuffix': '%', 'side': 'left'},
    xaxis12 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis13 = {'ticksuffix': '%', 'side': 'left'},
    xaxis13 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis14 = {'ticksuffix': '%', 'side': 'left'},
    xaxis14 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis15 = {'ticksuffix': '%', 'side': 'left'},
    xaxis15 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis16 = {'ticksuffix': '%', 'side': 'left'},
    xaxis16 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis17 = {'ticksuffix': '%', 'side': 'left'},
    xaxis17 = {
        'showgrid' : False
    }
 )
issuance_visual.fig.update_layout(
    yaxis18 = {'ticksuffix': '%', 'side': 'left'},
    xaxis18 = {
        'showgrid' : False
    }
 )

issuance_visual.fig.update_layout(
    showlegend=False,
    height= 500,
    width = 800,
    legend = {
#         'orientation': 'v',
#         'xanchor': 'left',
    'xanchor': 'center',
    'yanchor': 'bottom',
    'x': .5,
    'y': -.25,
    'orientation': 'h'
    })